# Versuch des Aufbaus eines Classifiers zur Vorhersage

Erneute Arbeit mit ORR-Daten.

Ich brauche Ground Truth Data als Ausgangspunkt. Dazu muss ich händisch einige Tweets in die Kategorien der Topic Models einordnen.



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import MultinomialNB
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset

In [7]:
train_df = pd.read_csv("mergerDaten/samples.csv")
train_df

,...1,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,...,ostdeutschland,politikWahlen,polizeiaktionen,streiks,ukraineInternational,ukrainekrieg,umweltKlima,verkehrsmeldung,wettermeldung,öpnv
0,24248,2022-07-18T15:54:02Z,1.549060e+18,Eine Frankfurter Pizzeria darf sich nach einem...,1,1,10,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,33169,2022-06-22T13:24:45Z,1.539600e+18,Die Verwüstung nach dem verheerenden Erdbeben ...,0,1,0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,38972,2022-02-22T12:51:15Z,1.496105e+18,"""Das Vorgehen Russlands ist völkerrechtswidrig...",11,8,37,2,NaN,NaN,...,0,1,0,0,0,1,0,0,0,0
3,32944,2022-06-28T04:21:04Z,1.541638e+18,Raketen-Angriff auf ein Einkaufszentrum im ukr...,4,2,11,0,NaN,NaN,...,0,0,1,0,0,1,0,0,1,0
4,29445,2022-09-16T13:51:36Z,1.570772e+18,In der Nordukraine versucht die ukrainische Ar...,1,1,3,0,NaN,NaN,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,30257,2022-08-29T07:23:05Z,1.564152e+18,Reallöhne sinken wegen hoher Inflation: Laut S...,4,10,15,2,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
146,73618,2022-04-08T10:02:42Z,1.512370e+18,Am Dienstag kommt die Sonne - mit bis zu 20 Gr...,0,0,16,2,NaN,NaN,...,0,0,0,0,0,0,1,0,1,1
147,36775,2022-04-05T10:12:24Z,1.511286e+18,In Deutschland gibt es in diesem Jahr wieder e...,3,2,3,0,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
148,94986,2022-04-03T07:00:15Z,1.510512e+18,"Ein Solarpark, der im Baggersee schwimmt. Den ...",8,18,111,3,NaN,NaN,...,1,1,0,0,0,0,0,0,0,0
